# Absorption résonante

# Une approche Monte Carlo de l'absorption résonante

Nous ne considérerons que la variable énergétique.  
En partant d'une source située en $E = 1$ Mev, modélisez via la méthode de Monte Carlo, le ralentissement dans un milieu ressemblant à l'hydrogène (collision élastique avec $A=1$) avec absorbant pur (dont la section efficace est négligeable en dehors de la résonance).   
Utilisez une source de 10 000 neutrons.  
Tracer le spectre en fonction de l'énergie.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field


@dataclass
class NuclearData:
    elastic_xs: float = 1.0
    capture_xs: float = 0.0
    energie_resonance: float = 0.5
    largeur_resonance: float = 0.01
    hauteur_resonance: float = 1e8
    atomic_weight_ratio: float = 1.0
    resonance_limits: tuple[float] = field(init=False)

    def __post_init__(self):
        self.resonance_limits = (
            self.energie_resonance - self.largeur_resonance / 2.0,
            self.energie_resonance + self.largeur_resonance / 2.0,
        )

    def get_xs(self, reaction_type, energy):
        if reaction_type.lower() == "elastic":
            return self._get_elastic_xs(energy)
        if reaction_type.lower() == "capture":
            return self._get_capture_xs(energy)
        if reaction_type.lower() == "total":
            return self._get_total_xs(energy)
        raise ValueError(
            f"Reaction {reaction_type} is not accepted. Accepted values are: elastic, capture or total"
        )

    def _get_elastic_xs(self, energy):
        return self.elastic_xs

    def _get_capture_xs(self, energy):
        if self.resonance_limits[0] < energy and energy < self.resonance_limits[1]:
            return self.hauteur_resonance
        else:
            return self.capture_xs

    def _get_total_xs(self, energy):
        return self._get_capture_xs(energy) + self._get_elastic_xs(energy)